In [1]:
import copy
import os
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
import peft
import torch
from torch import nn
import torch.nn.functional as F
from MLP_function import *


d:\ProgramData\Anaconda3\envs\FL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_all_seeds(45)
# 3D CNN
X = torch.rand((1000, 1, 28, 28, 28))
# set_all_seeds(45)
y = (X.sum(dim=[1,2,3,4]) > 11000).long()  # Binary classification based on voxel sum

print(y[0 :20])
print('y.sum',y.sum())

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
y.sum tensor(284)


In [3]:
n_train = 800
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X[:n_train], y[:n_train]),
    batch_size=batch_size,
    shuffle=True,
)
eval_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X[n_train:], y[n_train:]),
    batch_size=batch_size,
)
lr = 0.002
max_epochs = 30

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [4]:
# Lora
# set_all_seeds(45)
config = peft.LoraConfig(
    r=8,
    target_modules=["conv.0", "conv.3", 'fc.0'],
    # modules_to_save=["fc"] ,
)

original_model = CNN3D().to(device)
original_model_copy = copy.deepcopy(original_model)  # we keep a copy of the original model for later
# 计算所有参数拼接后的整体均值和方差
all_params = torch.cat([param.data.view(-1) for param in original_model.parameters()])
print(f"All params: mean= {all_params.mean().item():.6f}, std={all_params.std().item():.6f}")

All params: mean= 0.000007, std=0.006712


In [ ]:

# 3D CNN
## 
print('orginal_model type',type(original_model))
# print('orginal_model_copy type',type(original_model_copy))

print('before build peft model==========')
print('trainalbe parameter in orginal_model:')
for name, param in original_model.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")

# print("可训练参数列表:")
# for name, param in original_model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.shape}")

print('after build peft model==========')
print('orginal_model type',type(original_model))
peft_model = peft.get_peft_model(original_model_copy, config)
optimizer_peft = optim.Adam(peft_model.parameters(), lr=0.001)
print('trainalbe parameter in peft_model:')
for name, param in peft_model.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")

# print("可训练参数列表:")
# for name, param in peft_model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.shape}")

peft_model.print_trainable_parameters()

    

orginal_model type <class 'MLP_function.CNN3D'>
All params: mean= 0.000007, std=0.006712
before build peft model==========
trainalbe parameter in orginal_model:
Layer: conv.0.weight | Trainable: True
Layer: conv.0.bias | Trainable: True
Layer: conv.3.weight | Trainable: True
Layer: conv.3.bias | Trainable: True
Layer: fc.0.weight | Trainable: True
Layer: fc.0.bias | Trainable: True
Layer: fc.2.weight | Trainable: True
Layer: fc.2.bias | Trainable: True
after build peft model==========
orginal_model type <class 'MLP_function.CNN3D'>
All params: mean= 0.000007, std=0.006712
trainalbe parameter in peft_model:
Layer: base_model.model.conv.0.base_layer.weight | Trainable: False
Layer: base_model.model.conv.0.base_layer.bias | Trainable: False
Layer: base_model.model.conv.0.lora_A.default.weight | Trainable: True
Layer: base_model.model.conv.0.lora_B.default.weight | Trainable: True
Layer: base_model.model.conv.3.base_layer.weight | Trainable: False
Layer: base_model.model.conv.3.base_layer.

In [5]:
optimizer = optim.Adam(original_model.parameters(), lr=0.001)
criterion = nn.NLLLoss()
train(original_model, optimizer, criterion, train_dataloader, eval_dataloader, device,epochs=max_epochs)

Batch 0 get train loss: 0.6741485595703125 val loss: 0.5019506216049194
Batch 1 get train loss: 0.662207841873169 val loss: 0.5949137210845947
Batch 2 get train loss: 0.6080417633056641 val loss: 0.5036713480949402
Batch 3 get train loss: 0.6113957166671753 val loss: 0.5264621376991272
Batch 4 get train loss: 0.6091873645782471 val loss: 0.5454394221305847
Batch 5 get train loss: 0.6019322276115417 val loss: 0.5164607167243958
Batch 6 get train loss: 0.6136137843132019 val loss: 0.5475145578384399
Batch 7 get train loss: 0.6211646199226379 val loss: 0.5145511031150818
Batch 8 get train loss: 0.599980354309082 val loss: 0.5161627531051636
Batch 9 get train loss: 0.607366144657135 val loss: 0.5365462899208069
Batch 10 get train loss: 0.6193227767944336 val loss: 0.5075579881668091
Batch 11 get train loss: 0.6150994300842285 val loss: 0.5326802730560303
Batch 12 get train loss: 0.6088243722915649 val loss: 0.5208389163017273
Batch 13 get train loss: 0.6036936044692993 val loss: 0.52128058

In [ ]:
train(peft_model, optimizer_peft, criterion, train_dataloader, eval_dataloader, device,epochs=max_epochs)

In [ ]:
# Save model checkpoints
model_path = 'model_checkpoints/3DCNN_base'
if not os.path.exists(model_path):
    os.makedirs(model_path)
torch.save(original_model.state_dict(), os.path.join(model_path, f"model.pt"))

In [10]:
load_model_path = 'model_checkpoints/3DCNN_base/model.pt'
original_model_copy.load_state_dict(torch.load(load_model_path))
with torch.no_grad():
    y__loaded = original_model_copy(X.to(device))
    y_trained = original_model(X.to(device))
    
print('if the trained model and loaded model are the same:', torch.allclose(y__loaded, y_trained))
del y__loaded, y_trained
torch.cuda.empty_cache()

if the trained model and loaded model are the same: True


In [5]:
1e-3*0.99**(900/5)

0.00016380796970808743

In [ ]:
# Save lora adapter
adapter_save_path = 'lora_adapter/adapter_1'
peft_model.save_pretrained(adapter_save_path)

In [ ]:
# load adapter
model_name = 'lora_adapter/adapter_1'
peft_model_from_adapter = peft.PeftModel.from_pretrained(original_model_copy, model_name)
print('peft_model_from_adapter type',type(peft_model_from_adapter))
print('original_model_copy type',type(original_model_copy))
with torch.no_grad():
    y_peft = peft_model(X.to(device))
    y_loaded_adapter = peft_model_from_adapter(X.to(device))
print('if peft_model and peft_model_from_adapter have same results:',torch.allclose(y_peft, y_loaded_adapter))
del y_peft, y_loaded_adapter
torch.cuda.empty_cache()

peft_model_from_adapter type <class 'peft.peft_model.PeftModel'>
original_model_copy type <class 'MLP_function.CNN3D'>
if peft_model and peft_model_from_adapter have same results True


In [15]:
### Merging
model_name = 'lora_adapter/adapter_1'
unmerged_peft_model = peft.PeftModel.from_pretrained(original_model_copy, model_name)
print('before merging==========')
print('unmerged_peft_model type',type(unmerged_peft_model))
print('original_model_copy type',type(original_model_copy))
print('trainalbe parameter in unmerged_peft_model:')
for name, param in unmerged_peft_model.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")

print('after merging==========')

merged_peft_model = unmerged_peft_model.merge_and_unload()  
print('merged_peft_model type',type(merged_peft_model))
print('unmerged_peft_model type',type(unmerged_peft_model))

print('trainalbe parameter in merged_peft_model:')
for name, param in merged_peft_model.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")


before merging==========
unmerged_peft_model type <class 'peft.peft_model.PeftModel'>
original_model_copy type <class 'MLP_function.CNN3D'>
trainalbe parameter in unmerged_peft_model:
Layer: base_model.model.conv.0.base_layer.weight | Trainable: False
Layer: base_model.model.conv.0.base_layer.bias | Trainable: False
Layer: base_model.model.conv.0.lora_A.default.weight | Trainable: False
Layer: base_model.model.conv.0.lora_B.default.weight | Trainable: False
Layer: base_model.model.conv.3.base_layer.weight | Trainable: False
Layer: base_model.model.conv.3.base_layer.bias | Trainable: False
Layer: base_model.model.conv.3.lora_A.default.weight | Trainable: False
Layer: base_model.model.conv.3.lora_B.default.weight | Trainable: False
Layer: base_model.model.fc.0.base_layer.weight | Trainable: False
Layer: base_model.model.fc.0.base_layer.bias | Trainable: False
Layer: base_model.model.fc.0.lora_A.default.weight | Trainable: False
Layer: base_model.model.fc.0.lora_B.default.weight | Trainab

In [13]:
# Compare results of merged model and unmerged model
y_unmerged = unmerged_peft_model(X.to(device))
y_merged = merged_peft_model(X.to(device))
print('loRA model merged sucessfull:',torch.allclose(y_unmerged, y_merged))

loRA model merged sucessfull: True


In [16]:
y_module = module_copy(X.to(device))
print('check if merged model same as original model:',torch.allclose(y_module, y_merged))

check if merged model same as original model: True


In [14]:
print('module copy:',type(module_copy))

for name, param in module_copy.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")

print('\nmodule:',type(module))

for name, param in module.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")

module copy: <class 'MLP_function.CNN3D'>
Layer: conv.0.weight | Trainable: False
Layer: conv.0.bias | Trainable: False
Layer: conv.3.weight | Trainable: False
Layer: conv.3.bias | Trainable: False
Layer: fc.0.weight | Trainable: False
Layer: fc.0.bias | Trainable: False
Layer: fc.2.weight | Trainable: False
Layer: fc.2.bias | Trainable: False

module: <class 'MLP_function.CNN3D'>
Layer: conv.0.weight | Trainable: True
Layer: conv.0.bias | Trainable: True
Layer: conv.3.weight | Trainable: True
Layer: conv.3.bias | Trainable: True
Layer: fc.0.weight | Trainable: True
Layer: fc.0.bias | Trainable: True
Layer: fc.2.weight | Trainable: True
Layer: fc.2.bias | Trainable: True


In [15]:
# 1. 比较 state_dict 的 key 集合
orig_sd   = module_copy.state_dict()
merged_sd = merged_peft_model.state_dict()

orig_keys   = set(orig_sd.keys())
merged_keys = set(merged_sd.keys())

print("Key 集合完全相同：", orig_keys == merged_keys)
if orig_keys != merged_keys:
    print("仅在原 model 中有的 keys:", orig_keys - merged_keys)
    print("仅在 merged_model 中有的 keys:", merged_keys - orig_keys)

#    通常 merge 后参数会变化；如果你只是想确认结构，shape 检查到这里就够了。
for k in orig_keys & merged_keys:
    a = orig_sd[k]
    b = merged_sd[k]
    if not torch.equal(a, b):
        print(f"参数值不同: {k} （或使用 torch.allclose 验证近似相等）")
        # 如果想看具体差异，可以打印范数／最大差值
        diff = (a - b).abs()
        print(f"   max|Δ| = {diff.max():.3e}, mean|Δ| = {diff.mean():.3e}")
        # 只示例第一个不同的 key，然后跳出
        break
else:
    print("所有参数值完全相同！")

Key 集合完全相同： True
所有参数值完全相同！
